In [ ]:
!pip install stable-baselines3

In [ ]:
# Optional: install SB3 contrib to have access to additional algorithms
!pip install sb3-contrib

In [ ]:
# Optuna will be used in the last part when doing hyperparameter tuning
!pip install optuna

In [ ]:
import gym
import numpy as np

In [ ]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN

In [ ]:
# Algorithms from the contrib repo
# https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
from sb3_contrib import QRDQN, TQC

In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
env_id = "Pendulum-v1"
# Env used only for evaluation
eval_envs = make_vec_env(env_id, n_envs=10)
# 4000 training timesteps
budget_pendulum = 4000

In [ ]:
ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Define and train a A2C model
a2c_model = A2C("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
# Evaluate the train A2C model
mean_reward, std_reward = evaluate_policy(a2c_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"A2C Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# train longer
new_budget = 10 * budget_pendulum

ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(new_budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
tuned_params = {
        "gamma": 0.9,
        "use_sde": True,
        "sde_sample_freq": 4,
        "learning_rate": 1e-3,
        }

# budget = 10 * budget_pendulum
ppo_tuned_model = PPO("MlpPolicy", env_id, seed=1, verbose=1, **tuned_params).learn(50_000, log_interval=5)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_tuned_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"Tuned PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
budget = 20_000

In [ ]:
eval_envs_cartpole = make_vec_env("CartPole-v1", n_envs=10)

In [ ]:
model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=1).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
import torch.nn as nn

In [ ]:
policy_kwargs = dict(
            net_arch=[
                dict(vf=[64, 64], pi=[64, 64]), # network architectures for actor/critic
                ],
            activation_fn=nn.Tanh,
            )

hyperparams = dict(
          n_steps=5,        # number of steps to collect data before updating policy
          learning_rate=7e-4,
          gamma=0.99,        # discount factor
          max_grad_norm=0.5,    # The maximum value for the gradient clipping
          ent_coef=0.0,       # Entropy coefficient for the loss calculation
          )

model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=1, **hyperparams).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
gamma = trial.suggest_float("gamma", 0.9, 0.99999, log=True)
max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
# from 2**3 = 8 to 2**10 = 1024
n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
# net_arch tiny: {"pi": [64], "vf": [64]}
# net_arch default: {"pi": [64, 64], "vf": [64, 64]}
# activation_fn = nn.Tanh / nn.ReLU

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

In [ ]:
N_TRIALS = 100       # Maximum number of trials
N_JOBS = 1         # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5    # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2     # Number of evaluations during the training
N_TIMESTEPS = int(2e4)   # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15 minutes

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
            "policy": "MlpPolicy",
            "env": ENV_ID,
            }

In [ ]:
from typing import Any, Dict
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
  """
  Sampler for A2C hyperparameters.

  :param trial: Optuna trial object
  :return: The sampled hyperparameters for the given trial.
  """
  # Discount factor between 0.9 and 0.9999
  gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
  max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
  # 8, 16, 32, ... 1024
  n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

  ### YOUR CODE HERE
  # TODO:
  # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
  # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
  # - define the activation function search space ["tanh", "relu"]
  learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
  net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
  activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

  ### END OF YOUR CODE

  # Display true values
  trial.set_user_attr("gamma_", gamma)
  trial.set_user_attr("n_steps", n_steps)

  net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
        if net_arch == "small"
        else trial.suggest_categorical("net_arch", [{"pi": [64, 64], "vf": [64, 64]}], )
        ]

  activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

  return {
      "n_steps": n_steps,
      "gamma": gamma,
      "learning_rate": learning_rate,
      "max_grad_norm": max_grad_norm,
      "policy_kwargs": {
                "net_arch": net_arch,
                "activation_fn": activation_fn,
               },
      }

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
  """
  Callback used for evaluating and reporting a trial.

  :param eval_env:      Evaluation environement
  :param trial:        Optuna trial object
  :param n_eval_episodes:   Number of evaluation episodes
  :param eval_freq:      Evaluate the agent every ``eval_freq`` call of the callback.
  :param deterministic:    Whether the evaluation should use a stochastic or deterministic policy.
  :param verbose:
  """

  def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
        ):

    super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
            )
    self.trial = trial
    self.eval_idx = 0
    self.is_pruned = False

  def _on_step(self) -> bool:
    if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
      # Evaluate policy (done in the parent class)
      super()._on_step()
      self.eval_idx += 1
      # Send report to Optuna
      self.trial.report(self.last_mean_reward, self.eval_idx)
      # Prune trial if need
      if self.trial.should_prune():
        self.is_pruned = True
        return False
    return True

In [ ]:
def objective(trial: optuna.Trial) -> float:
  """
  Objective function using by Optuna to evaluate
  one configuration (i.e., one set of hyperparameters).

  Given a trial object, it will sample hyperparameters,
  evaluate it and report the result (mean episodic reward after training)

  :param trial: Optuna trial object
  :return: Mean episodic reward after training
  """

  kwargs = DEFAULT_HYPERPARAMS.copy()
  ### YOUR CODE HERE
  # TODO:
  # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
  # 2. Create the evaluation envs
  # 3. Create the `TrialEvalCallback`

  # 1. Sample hyperparameters and update the keyword arguments
  kwargs.update(sample_a2c_params(trial))

  # Create the RL model
  model = A2C(**kwargs)

  # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
  eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS)

  # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
  # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
  # TrialEvalCallback signature:
  # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
  eval_callback = TrialEvalCallback(
                    eval_envs,
                    trial,
                    n_eval_episodes=N_EVAL_EPISODES,
                    eval_freq=EVAL_FREQ,
                    deterministic=True,
                    verbose=1,
                    )

  ### END OF YOUR CODE

  nan_encountered = False
  try:
    # Train the model
    model.learn(N_TIMESTEPS, callback=eval_callback)
  except AssertionError as e:
    # Sometimes, random hyperparams can generate NaN
    print(e)
    nan_encountered = True
  finally:
    # Free memory
    model.env.close()
    eval_envs.close()

  # Tell the optimizer that the trial failed
  if nan_encountered:
    return float("nan")

  if eval_callback.is_pruned:
    raise optuna.exceptions.TrialPruned()

  return eval_callback.last_mean_reward

In [ ]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
            n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
            )
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
  study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
  pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
  print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
  print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()